Прежде чем отправиться в путь, вы должны рассмотреть ваш вариант использования. При взаимодействии с нативным кодом обычно возникает два варианта использования:

- Существующий код на C / C++, который необходимо использовать либо потому, что он уже существует, либо потому, что он быстрее.
- Код Python слишком медленный, вставьте внутренние циклы в нативный код

Каждая технология демонстрируется упаковкой функции cos из math.h. Хотя это, в основном, тривиальный пример, он должен хорошо послужить нам для демонстрации основ решения обертывания. Поскольку каждая техника также включает некоторую форму поддержки Numpy, это также демонстрируется на примере, где косинус вычисляется для некоторого вида массива.

Последнее, но не менее важное, два небольших предупреждения:

- Все эти методы могут вызвать сбой (ошибка сегментации) интерпретатора Python, что (как правило) связано с ошибками в коде C.
- Все примеры были сделаны на Linux, они должны быть возможны на других операционных системах.
- Вам понадобится компилятор C для большинства примеров.

### Python-C-Api

Python-C-API является основой стандартного интерпретатора Python (a.k.a CPython). Используя этот API, можно написать модуль расширения Python на C и C++. Очевидно, что эти модули расширения могут, в силу языковой совместимости, вызывать любую функцию, написанную на C или C++.

При использовании Python-C-API обычно пишется много стандартного кода, сначала для анализа аргументов, которые были переданы функции, а затем для создания возвращаемого типа.

**Преимущества**

- Не требует дополнительных библиотек
- Много контроля низкого уровня
- Полностью используется из C++

**Недостатки**

- Может потребовать значительных усилий
- Много накладных расходов в коде
- Должен быть скомпилирован
- Высокая стоимость обслуживания
- Отсутствует прямая совместимость между версиями Python при изменении C-API
- Ошибки подсчета ссылок легко создавать, и их очень трудно отследить.

**Официальная документация**: https://docs.python.org/3.8/c-api/intro.html#objects-types-and-reference-counts

Следующий модуль C-extension делает функцию cos из стандартной математической библиотеки доступной для Python:

```C
/*  Example of wrapping cos function from math.h with the Python-C-API. */

#include <Python.h>
#include <math.h>

/*  wrapped cosine function */
static PyObject* cos_func(PyObject* self, PyObject* args)
{
    double value;
    double answer;

    /*  parse the input, from python float to c double */
    if (!PyArg_ParseTuple(args, "d", &value))
        return NULL;
    /* if the above function returns -1, an appropriate Python exception will
     * have been set, and the function simply returns NULL
     */

    /* call cos from libm */
    answer = cos(value);

    /*  construct the output from cos, from c double to python float */
    return Py_BuildValue("f", answer);
}

/*  define functions in module */
static PyMethodDef CosMethods[] =
{
     {"cos_func", cos_func, METH_VARARGS, "evaluate the cosine"},
     {NULL, NULL, 0, NULL}
};

#if PY_MAJOR_VERSION >= 3
/* module initialization */
/* Python version 3*/
static struct PyModuleDef cModPyDem =
{
    PyModuleDef_HEAD_INIT,
    "cos_module", "Some documentation",
    -1,
    CosMethods
};

PyMODINIT_FUNC
PyInit_cos_module(void)
{
    return PyModule_Create(&cModPyDem);
}

#else

/* module initialization */
/* Python version 2 */
PyMODINIT_FUNC
initcos_module(void)
{
    (void) Py_InitModule("cos_module", CosMethods);
}

#endif
```

Как вы можете видеть, существует много шаблонного, как для «массирования» аргументов и возвращаемых типов, так и для инициализации модуля. Хотя часть этого амортизируется по мере роста расширения, шаблон, необходимый для каждой функции, остается.

Стандартная система сборки python distutils поддерживает компиляцию C-расширений из setup.py, что довольно удобно:

```Python
from distutils.core import setup, Extension

# define the extension module
cos_module = Extension('cos_module', sources=['cos_module.c'])

# run the setup
setup(ext_modules=[cos_module])
```

Компиляция:

- `build_ext` для сборки модулей расширения
- `--inplace` выведет скомпилированный модуль расширения в текущий каталог

Файл `cos_module.so` содержит скомпилированное расширение, которое мы теперь можем загрузить в интерпретаторе IPython:
```Python
import cos_module
cos_module?
dir(cos_module)
cos_module.cos_func(1.0)
```

1. [Прототип лабораторной работы по созданию модулей на Си](https://docs.google.com/document/d/1lXbsgWJUiGKM8jn_npiei0vVjDZwJsC7xcgp6W7Q_ss/edit?usp=sharing)

2. [Building a Python C Extension Module](https://realpython.com/build-python-c-extension-module/)

### NumPy API

По аналогии с Python-C-API, Numpy, который сам реализован как расширение C, поставляется с [Numpy-C-API](https://docs.scipy.org/doc/numpy/reference/c-api.html). Этот API может использоваться для создания и управления массивами Numpy из C при написании собственного C-расширения. 

В следующем примере показано, как передавать массивы Numpy в качестве аргументов в функции и как перебирать массивы Numpy с помощью (старого) Numpy-C-API. Он просто принимает массив в качестве аргумента, применяет функцию косинуса из math.h и возвращает полученный новый массив.

```C
/*  Example of wrapping the cos function from math.h using the Numpy-C-API. */

#include <Python.h>
#include <numpy/arrayobject.h>
#include <math.h>

/*  wrapped cosine function */
static PyObject* cos_func_np(PyObject* self, PyObject* args)
{

    PyArrayObject *in_array;
    PyObject      *out_array;
    NpyIter *in_iter;
    NpyIter *out_iter;
    NpyIter_IterNextFunc *in_iternext;
    NpyIter_IterNextFunc *out_iternext;

    /*  parse single numpy array argument */
    if (!PyArg_ParseTuple(args, "O!", &PyArray_Type, &in_array))
        return NULL;

    /*  construct the output array, like the input array */
    out_array = PyArray_NewLikeArray(in_array, NPY_ANYORDER, NULL, 0);
    if (out_array == NULL)
        return NULL;

    /*  create the iterators */
    in_iter = NpyIter_New(in_array, NPY_ITER_READONLY, NPY_KEEPORDER,
                             NPY_NO_CASTING, NULL);
    if (in_iter == NULL)
        goto fail;

    out_iter = NpyIter_New((PyArrayObject *)out_array, NPY_ITER_READWRITE,
                          NPY_KEEPORDER, NPY_NO_CASTING, NULL);
    if (out_iter == NULL) {
        NpyIter_Deallocate(in_iter);
        goto fail;
    }

    in_iternext = NpyIter_GetIterNext(in_iter, NULL);
    out_iternext = NpyIter_GetIterNext(out_iter, NULL);
    if (in_iternext == NULL || out_iternext == NULL) {
        NpyIter_Deallocate(in_iter);
        NpyIter_Deallocate(out_iter);
        goto fail;
    }
    double ** in_dataptr = (double **) NpyIter_GetDataPtrArray(in_iter);
    double ** out_dataptr = (double **) NpyIter_GetDataPtrArray(out_iter);

    /*  iterate over the arrays */
    do {
        **out_dataptr = cos(**in_dataptr);
    } while(in_iternext(in_iter) && out_iternext(out_iter));

    /*  clean up and return the result */
    NpyIter_Deallocate(in_iter);
    NpyIter_Deallocate(out_iter);
    Py_INCREF(out_array);
    return out_array;

    /*  in case bad things happen */
    fail:
        Py_XDECREF(out_array);
        return NULL;
}

/*  define functions in module */
static PyMethodDef CosMethods[] =
{
     {"cos_func_np", cos_func_np, METH_VARARGS,
         "evaluate the cosine on a numpy array"},
     {NULL, NULL, 0, NULL}
};


#if PY_MAJOR_VERSION >= 3
/* module initialization */
/* Python version 3*/
static struct PyModuleDef cModPyDem = {
    PyModuleDef_HEAD_INIT,
    "cos_module", "Some documentation",
    -1,
    CosMethods
};
PyMODINIT_FUNC PyInit_cos_module_np(void) {
    PyObject *module;
    module = PyModule_Create(&cModPyDem);
    if(module==NULL) return NULL;
    /* IMPORTANT: this must be called */
    import_array();
    if (PyErr_Occurred()) return NULL;
    return module;
}

#else
/* module initialization */
/* Python version 2 */
PyMODINIT_FUNC initcos_module_np(void) {
    PyObject *module;
    module = Py_InitModule("cos_module_np", CosMethods);
    if(module==NULL) return;
    /* IMPORTANT: this must be called */
    import_array();
    return;
}

#endif
```

Для компиляции мы можем снова использовать distutils. Однако мы должны обязательно включать заголовки Numpy с помощью: func: numpy.get_include.

```Python
from distutils.core import setup, Extension
import numpy

# define the extension module
cos_module_np = Extension('cos_module_np', sources=['cos_module_np.c'],
                          include_dirs=[numpy.get_include()])

# run the setup
setup(ext_modules=[cos_module_np])
```

Чтобы убедиться, что это действительно работает, мы запускаем следующий тестовый скрипт:

```Python
import cos_module_np
import numpy as np
import matplotlib.pyplot as plt

x = np.arange(0, 2 * np.pi, 0.1)
y = cos_module_np.cos_func_np(x)
plt.plot(x, y)
plt.show()
```

Про Cython: http://scipy-lectures.org/advanced/interfacing_with_c/interfacing_with_c.html#id10